# Logbook and all things for Monte-Carlo Minimizor-Maximization

# Algorithm

1. Monte-Carlo Descent method to search for optimal point
  * Use a gradient-based local optimization method 
    * This requires auto-grad by Jax/PyTorch/TensorFlow.
    * Use Jax for now.
  * Use MCTD to balance exploration and exploitation of the search space
2. Learn the lower bound of the objective function
  * Use a quadratic model to approximate the lower bound of the objective function
  * The learning is as a linear regression problem: finding `A`, such that `Ax <= y`
3. Use interval computation to estimate if the optimal point is found within a small box
  * Use ibex to compute the interval of the objective function 
4. Use branch-and-bound to split the search space into smaller boxes
  * Better approx -> better pruning 
  * Prunning ambig box -> high priority 

### Problems to solve

Use an toy example  
$\min( 1000*(x+1)^2-3, (x-1)^2-1)$

* During MCTD optimization, we have
  * A list of samples from each trajectory
  * A best-found point (x=1) from 1 or more nodes (but it is not the global min)
  * 0 or more local best-found from 1 or more nodes

3. Use interval computation to estimate if the optimal point is found within a small box  
  * interval + function -> Ibex -> interval (can't tell directly when the formula is sat)
    * Use IBEX to find a small box B' that $f == lb_f$ may occur in
  * Better LB, better pruning?
    * IBEX works better with quadratic instead of linear. 
    * The best lb to learn is a function that is NEITHER tightly lb to original function NOR completely flat
  * Learned small box B', restart by another MCTD?
    * No. Sample within the box for another node on MCTD
    * We need a global LB for the original function, not a piecewise LB.



# Benchmark Settings

## Datasets

1. Synthetic functions
  * [SFU](https://www.sfu.ca/~ssurjano/optimization.html)
  * [Kyoto](http://www-optima.amp.i.kyoto-u.ac.jp/member/student/hedar/Hedar_files/TestGO_files/Page364.htm)

2. Coconut Project (Maybe too old?)
  * [Coconut](https://arnold-neumaier.at/glopt/coconut/Benchmark/Benchmark.html)

3. Real Large Scale Global Optimization
  * [CEC](https://github.com/P-N-Suganthan/2022-SO-BO/blob/main/CEC2022%20TR.pdf)

4. Engineering problems
  * [Matlab](https://www.mathworks.com/matlabcentral/fileexchange/124810-benchmark-problems?s_tid=FX_rc1_behav)

5. Optimization of Black-Box Functions
  * [Black-Box](https://github.com/christiangeissler/gradoptbenchmark/tree/master/)

6. Sets of test problems for MINLP
  * [MINLP](https://www.minlp.com/nlp-and-minlp-test-problems)

## Baselines

1. [Gurobi]() - installed
2. [Scipy.optimize](https://docs.scipy.org/doc/scipy/reference/optimize.html) - installed
3. [PyGMO](https://esa.github.io/pagmo2/docs/cpp/algorithms/nlopt.html) - TODO 
4. [BARON](https://sahinidis.coe.gatech.edu/baron) - can do via [NEOS](https://neos-server.org/neos/) 
  * BARON guarantees to provide global optima under fairly general assumptions, but cannot handle constraints containing a goniometric function, an if-then-else statement or a reference to an external function. BARON requires that all nonlinear variables and expressions in the mathematical program are bounded from below and above. [source](https://documentation.aimms.com/platform/solvers/baron.html)






4. Gradient based ? check if there are in `scipy.optimize` or `pygmo`
5. Simulated Annealing ? check if there are in `scipy.optimize` or `pygmo`

10. [Matlab FMinCon](https://www.mathworks.com/help/optim/ug/fmincon.html) - installed, but least priority because it is not free and not in python
